In [4]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DataFrameLoader
import sentence_transformers
from langchain.vectorstores import Chroma
import pandas as pd

/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13108360
      0 drwxr-xr-x   14 kariato  staff   448B Feb 12 01:05 ./
      0 drwxr-xr-x    7 kariato  staff   224B Feb 10 21:29 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
      0 drwxr-xr-x    4 kariato  staff   128B Feb 12 02:29 chroma_db/
2785400 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
2109312 -rw-r--r--    1 kariato  staff   1.0G Feb 10 14:24 enron.sql
 427936 -rw-r--r--@   1 kariato  staff   209M Feb 10 14:24 enron.sql.gz
 865736 -rw-r--r--@   1 kariato  staff   423M Feb 10 14:35 enron_mail_20150507.tar.gz
6177656 -rw-r--r--    1 kariato  staff   2.9G Feb 11 23:50 enron_processed.csv
      0 drwxr-xr-x@  11 kariato  staff   352B Nov 10  2004 enron_with_categories/
   8840 -rw-r--r--@   1 kariato  staff   4.3M 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
enron_df=pd.read_csv('enron_processed.csv')

In [5]:
enron_df.drop(['body','raw'],axis=1,inplace=True)

In [6]:
enron_df.columns

Index(['Unnamed: 0', 'MESSAGE-ID', 'DATE', 'FROM', 'TO', 'SUBJECT',
       'MIME-VERSION', 'CONTENT-TYPE', 'CONTENT-TRANSFER-ENCODING',
       'body_short', 'CC', 'SENT', 'BCC'],
      dtype='object')

In [7]:
load_df = DataFrameLoader(enron_df, page_content_column='body_short')
documents = load_df.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [9]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 823kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 14.9MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 229kB/s]
config.json: 100%|██████████| 612/612 [00:00<00:00, 2.16MB/s]
/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:08<00:00, 10.4MB/s]
/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/torch/_utils.py:841: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 2.97MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 3.81MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 10.9MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 417kB/s]


In [10]:
db = Chroma.from_documents(docs, embedding_function,persist_directory="./chroma_db")

In [39]:
query = "illegal manipluation"
docs = db.similarity_search(query)

# print results
print(docs[2])

page_content="Subject: Re: Socal\n\nCorporate schizophrenia.  Weird.   DF\n\n\n   \n\t\n\t\n\t\n\n\nSubject: Socal\n\nSoCal's protest is ludicrous. I will work on a response.\nBy the way, SoCal has actually been remarkably helpful in reaching agreement \non a settlement -- which we just signed -- in the California GIR proceeding." metadata={'DATE': ' Mon, 17 Apr 2000 09:52', 'FROM': '>>   Susan Scott                           04/17/2000 03', 'SUBJECT': ' Re>> ', 'TO': '>> ', 'Unnamed: 0': 115182}


In [40]:
print(enron_df[enron_df['Unnamed: 0']==115182]['raw'].to_list()[0])

Message-ID: <12645594.1075842517758.JavaMail.evans@thyme>
Date: Mon, 17 Apr 2000 09:52:00 -0700 (PDT)
From: drew.fossum@enron.com
To: susan.scott@enron.com
Subject: Re: Socal
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Drew Fossum
X-To: Susan Scott
X-cc: 
X-bcc: 
X-Folder: \Drew_Fossum_Dec2000_June2001_1\Notes Folders\'sent mail
X-Origin: FOSSUM-D
X-FileName: dfossum.nsf

Corporate schizophrenia.  Weird.   DF


   
	
	
	From:  Susan Scott                           04/17/2000 03:17 PM
	

To: Drew Fossum@ENRON
cc:  

Subject: Socal

SoCal's protest is ludicrous. I will work on a response.
By the way, SoCal has actually been remarkably helpful in reaching agreement 
on a settlement -- which we just signed -- in the California GIR proceeding.




In [28]:
print(enron_df[enron_df['Unnamed: 0']==32653]['body'].to_list()[0])

Subject: Energy market outlook 11/27/01

